In [1]:
import os

model_root = '../results/models'
files = os.listdir(model_root)
params = ['_'.join(f.split('_')[2:]) for f in files]
params = [p.split('.pkl')[0] for p in params]
params = list(set(params))
params

['p_0.8055551041134607_q_0.1_g_1',
 'p_1.0795506927238254_q_8.383911078685804_g_1',
 'p_0.5_q_1.895944090041435_g_1',
 'p_19.0_q_8.483911078685804_g_2',
 'p_7.305688086564288_q_7.517332462471247_g_2',
 'p_19.0_q_9.122152261131532_g_1']

In [2]:
import pandas as pd
import pickle

for p in params:
    emb_fp = f'../data/emb/from_adelle/emb_{p}.tsv'
    emb = pd.read_csv(emb_fp, sep='\t', index_col=0)
    samples = emb[emb.index.str.startswith('MD')]
    end_samples = samples[samples.index.str.contains('End')]
    features = emb[~emb.index.str.startswith('MD')]
    for classifier_type, colnames in zip(['time', 'diet'], [['baseline', 'endpoint'], ['dairy', 'meat']]):
        for model_num in range(1,6):
            model_fp = f'{model_root}/{classifier_type}_{model_num}_{p}.pkl'
            if not os.path.exists(model_fp):
                continue
            with open(model_fp, 'rb') as f:
                model = pickle.load(f)
            feature_preds = model.predict_proba(features)
            feature_preds = pd.DataFrame(feature_preds, index=features.index, columns=colnames)
            if classifier_type == 'time':
                sample_preds = model.predict_proba(samples)
                sample_preds = pd.DataFrame(sample_preds, index=samples.index, columns=colnames)

            else:
                sample_preds = model.predict_proba(end_samples)
                sample_preds = pd.DataFrame(sample_preds, index=end_samples.index, columns=colnames)

            #feature_preds.to_csv(f'/mnt/home/f0106093/Projects/multiomics-embedding/results/predictions/{classifier_type}_{model_num}_{p}_features.tsv', sep='\t')
            #sample_preds.to_csv(f'/mnt/home/f0106093/Projects/multiomics-embedding/results/predictions/{classifier_type}_{model_num}_{p}_samples.tsv', sep='\t')

/mnt/home/f0106093/anaconda3/envs/multiomics/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/mnt/home/f0106093/anaconda3/envs/multiomics/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/mnt/home/f0106093/anaconda3/envs/multiomics/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/mnt/home/f0106093/anaconda3/envs/multiomics/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegre

In [3]:
root = '../results/predictions'

def get_predictions(file):
    if os.path.exists(file):
        df = pd.read_csv(file, sep='\t', index_col=0)
        return df[df.columns[0]].to_list()

diet_features = {}
time_features = {}
diet_samples = {}
time_samples = {}

for p in params:
    diet_features[p] = {}
    time_features[p] = {}
    diet_samples[p] = {}
    time_samples[p] = {}
    for model_num in range(1,6):
        diet_features[p][model_num] = get_predictions(f'{root}/diet_{model_num}_{p}_features.tsv')
        time_features[p][model_num] = get_predictions(f'{root}/time_{model_num}_{p}_features.tsv')
        diet_samples[p][model_num] = get_predictions(f'{root}/diet_{model_num}_{p}_samples.tsv')
        time_samples[p][model_num] = get_predictions(f'{root}/time_{model_num}_{p}_samples.tsv')




In [4]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

def get_p_val(df, col1, col2):
    if col1 == col2:
        return 0
    else:
        corr, p_value = spearmanr(df[col1], df[col2])
        return p_value


def make_heatmap(correlation_matrix, classifier_type, data_type, correlation_type):
    plt.figure(figsize=(12, 10))
    cmap = sns.diverging_palette(240, 10, as_cmap=True)
    sns.heatmap(correlation_matrix, cmap=cmap, center=0, vmin=-1, vmax=1, fmt=".2f", xticklabels=True, yticklabels=True, annot=True)
    plt.title(f'{classifier_type} {data_type} {correlation_type}')
    correlation_type = correlation_type.replace(' ', '_')
    plt.savefig(f'../results/correlations/heatmaps/{classifier_type}_{data_type}_{correlation_type}.png')
    plt.close()
    
def get_val_correlation(predictions, classifier_type, data_type):
    val_cors = {}
    for params in predictions.keys():
        preds = pd.DataFrame(predictions[params])
        correlation_matrix = preds.corr(method='spearman')
        p_vals = pd.DataFrame([(col1, col2, get_p_val(preds, col1, col2)) 
                       for col1 in preds.columns for col2 in preds.columns], 
                      columns=['model_1', 'model_2', 'p_value'])
        p_vals = p_vals.pivot(index='model_1', columns='model_2', values='p_value')
        p_vals.to_csv(f'../results/correlations/{classifier_type}_{data_type}_{params}_p_vals.tsv', sep='\t')
        correlation_matrix.to_csv(f'../results/correlations/{classifier_type}_{data_type}_{params}.tsv', sep='\t')
        make_heatmap(correlation_matrix, classifier_type, data_type, f'within embedding correlations {params}')
        corrs = np.triu(correlation_matrix, k=1)
        corrs = corrs[corrs > 0].flatten()
        val_cors[params] = np.mean(corrs)
    return val_cors


diet_features_val_cors = get_val_correlation(diet_features, 'diet', 'features')
time_features_val_cors = get_val_correlation(time_features, 'time', 'features')
diet_samples_val_cors = get_val_correlation(diet_samples, 'diet', 'samples')
time_samples_val_cors = get_val_correlation(time_samples, 'time', 'samples')
    

In [5]:
def get_avg_predictions(predictions):
    avg_preds = {}
    for params in predictions.keys():
        preds = pd.DataFrame(predictions[params])
        avg_preds[params] = preds.mean(axis=1).to_list()
    return avg_preds

def get_avg_param_correlation(predictions, classifier_type, data_type):
    avg_corrs = {}
    avg_preds = pd.DataFrame(get_avg_predictions(predictions))
    correlation_matrix = avg_preds.corr(method='spearman')
    p_vals = pd.DataFrame([(col1, col2, get_p_val(avg_preds, col1, col2)) 
                       for col1 in avg_preds.columns for col2 in avg_preds.columns], 
                      columns=['emb_1', 'emb_2', 'p_value'])
    p_vals = p_vals.pivot(index='emb_1', columns='emb_2', values='p_value')
    p_vals.to_csv(f'../results/correlations/avg_{classifier_type}_{data_type}_p_vals.tsv', sep='\t')
    make_heatmap(correlation_matrix, classifier_type, data_type, 'average between embedding correlation')
    corrs = pd.DataFrame(np.triu(correlation_matrix, k=1), index=correlation_matrix.index, columns=correlation_matrix.columns)
    correlation_matrix.to_csv(f'../results/correlations/avg_{classifier_type}_{data_type}.tsv', sep='\t')
    for row in corrs.index:
        for col in corrs.columns:
            if corrs.loc[row, col] > 0:
                avg_corrs[f'{row}+{col}'] = corrs.loc[row, col]
    print(avg_corrs)
    return avg_corrs

diet_features_avg_corr = get_avg_param_correlation(diet_features, 'diet', 'features')
time_features_avg_corr = get_avg_param_correlation(time_features, 'time', 'features')
diet_samples_avg_corr = get_avg_param_correlation(diet_samples, 'diet', 'samples')
time_samples_avg_corr = get_avg_param_correlation(time_samples, 'time', 'samples')

{'p_0.8055551041134607_q_0.1_g_1+p_1.0795506927238254_q_8.383911078685804_g_1': 0.37994590593303923, 'p_0.8055551041134607_q_0.1_g_1+p_0.5_q_1.895944090041435_g_1': 0.18361177532973563, 'p_0.8055551041134607_q_0.1_g_1+p_19.0_q_8.483911078685804_g_2': 0.28188869066233574, 'p_0.8055551041134607_q_0.1_g_1+p_7.305688086564288_q_7.517332462471247_g_2': 0.26889809094856515, 'p_0.8055551041134607_q_0.1_g_1+p_19.0_q_9.122152261131532_g_1': 0.2629639395274258, 'p_1.0795506927238254_q_8.383911078685804_g_1+p_0.5_q_1.895944090041435_g_1': 0.17621972284369003, 'p_1.0795506927238254_q_8.383911078685804_g_1+p_19.0_q_8.483911078685804_g_2': 0.1809058336505104, 'p_1.0795506927238254_q_8.383911078685804_g_1+p_7.305688086564288_q_7.517332462471247_g_2': 0.19255888758189907, 'p_1.0795506927238254_q_8.383911078685804_g_1+p_19.0_q_9.122152261131532_g_1': 0.18987946892959168, 'p_0.5_q_1.895944090041435_g_1+p_19.0_q_8.483911078685804_g_2': 0.10185306261403874, 'p_0.5_q_1.895944090041435_g_1+p_7.3056880865642

In [6]:
def get_model_correlation(predictions, classifier_type, data_type):
    model_corrs = {}
    for model in range(1,6):
        preds = []
        for params in predictions.keys():
            preds.append(predictions[params][model])
        preds = pd.DataFrame(preds, index=predictions.keys()).T
        correlation_matrix = preds.corr(method='spearman')
        p_vals = pd.DataFrame([(col1, col2, get_p_val(preds, col1, col2)) 
                       for col1 in preds.columns for col2 in preds.columns], 
                      columns=['emb_1', 'emb_2', 'p_value'])
        p_vals = p_vals.pivot(index='emb_1', columns='emb_2', values='p_value')
        p_vals.to_csv(f'../results/correlations/{classifier_type}_{data_type}_model_{model}_p_vals.tsv', sep='\t')
        make_heatmap(correlation_matrix, classifier_type, data_type, f'model {model} between embedding correlation')
        correlation_matrix.to_csv(f'../results/correlations/{classifier_type}_{data_type}_model_{model}.tsv', sep='\t')
        corrs = np.triu(correlation_matrix, k=1)
        corrs = corrs[corrs > 0].flatten()
        model_corrs[model] = np.mean(corrs)
    return model_corrs

diet_features_model_corr = get_model_correlation(diet_features, 'diet', 'features')
time_features_model_corr = get_model_correlation(time_features, 'time', 'features')
diet_samples_model_corr = get_model_correlation(diet_samples, 'diet', 'samples')
time_samples_model_corr = get_model_correlation(time_samples, 'time', 'samples')
diet_features_model_corr, time_features_model_corr, diet_samples_model_corr, time_samples_model_corr

({1: 0.1399665241017364,
  2: 0.13519370033077147,
  3: 0.16407670588999185,
  4: 0.17546420771180415,
  5: 0.1496966387255825},
 {1: 0.2691829101730028,
  2: 0.2826835567542878,
  3: 0.30793865853455976,
  4: 0.3045820993809579,
  5: 0.2630383007180043},
 {1: 0.8742024531147727,
  2: 0.8367511881385356,
  3: 0.8580153106235237,
  4: 0.8354193346424201,
  5: 0.8442414411337829},
 {1: 0.9029852037191487,
  2: 0.8823463997775925,
  3: 0.8984635344268371,
  4: 0.8960238470330214,
  5: 0.8684570475396164})

In [7]:
sample_models = pd.DataFrame({'diet': diet_samples_model_corr, 'time': time_samples_model_corr })
feature_models = pd.DataFrame({'diet': diet_features_model_corr, 'time': time_features_model_corr })
samples_avg = pd.DataFrame({'diet': diet_samples_avg_corr, 'time': time_samples_avg_corr })
features_avg = pd.DataFrame({'diet': diet_features_avg_corr, 'time': time_features_avg_corr })
sample_models, feature_models, samples_avg, features_avg

(       diet      time
 1  0.874202  0.902985
 2  0.836751  0.882346
 3  0.858015  0.898464
 4  0.835419  0.896024
 5  0.844241  0.868457,
        diet      time
 1  0.139967  0.269183
 2  0.135194  0.282684
 3  0.164077  0.307939
 4  0.175464  0.304582
 5  0.149697  0.263038,
                                                         diet      time
 p_0.8055551041134607_q_0.1_g_1+p_1.079550692723...  0.814907  0.889519
 p_0.8055551041134607_q_0.1_g_1+p_0.5_q_1.895944...  0.824469  0.869317
 p_0.8055551041134607_q_0.1_g_1+p_19.0_q_8.48391...  0.820285  0.877806
 p_0.8055551041134607_q_0.1_g_1+p_7.305688086564...  0.806284  0.872514
 p_0.8055551041134607_q_0.1_g_1+p_19.0_q_9.12215...  0.788355  0.857687
 p_1.0795506927238254_q_8.383911078685804_g_1+p_...  0.862375  0.934121
 p_1.0795506927238254_q_8.383911078685804_g_1+p_...  0.866302  0.937846
 p_1.0795506927238254_q_8.383911078685804_g_1+p_...  0.856911  0.926624
 p_1.0795506927238254_q_8.383911078685804_g_1+p_...  0.896269  0.932175
 p